In [25]:
import scipy.stats as sts
import math
%matplotlib inline

import os
import sys
import glob
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline
plt.style.use('ggplot')
np.random.seed(1234)

np.set_printoptions(formatter={'all':lambda x: '%.10f' % x})

In [102]:
text_file = open("d://tmp//base_2.txt", "r")
lines = text_file.readlines()
text_file.close()

In [3]:
data = []
for i in lines:
    data.append(float(i))

In [8]:
k=10

In [12]:
n=len(data)

In [15]:
def set_initial_guess(xdata, k):
    np.random.seed(1234)
# set pi mu sigma
    pis = np.random.random(k)
    pis *= 0.9
    pis += 0.1
    pis /= np.sum(pis)
# mu
    mus = np.zeros(k)
# sigma
    sigmas = np.random.random(k)
    sigmas *= 1.5
    sigmas += 0.25
    sigmas *= np.std(xdata)
    return pis, mus, sigmas


In [115]:
pis, mus, sigmas = set_initial_guess(data, k)
for i in range (10):
    print('EM iteration {0}'.format(i))
    # E step
    ws = np.zeros((n, k))
    for j in range(k):
        for i in range(n):
            if sigmas[j] != 0:
                ws[i, j] = pis[j] * sts.norm(mus[j], sigmas[j]).pdf(data[i])
            #elif mus[j]-data[i] == 0:
            #    ws[i, j] = pis[j]
            #else:
            #    ws[i, j] = 0
    #S step
    ys = np.zeros((n, k))
    for i in range(n):
        ys[i] = np.random.multinomial(1, ws[i,:]/sum(ws[i,:]), size=1)

    v = ys.sum(0)

    #M step
    ss=[]
    for i in range(k):
        ss.append([])
    for i in range(n):
        for j in range(k):
            if (ys[i,j] == 1):
                ss[j].append(data[i])
    for j in range(k):
        ss[j].sort()
    print(v)
    #pi
    pis = v/n
    #mu
    mus = np.zeros(k)
    for j in range(k):
        if (v[j] %2 == 0):
            mus[j]=0.5*(ss[j][int(v[j])//2-1]+ss[j][int(v[j])//2])
        else:
            print(j)
            mus[j]=ss[j][math.floor(v[j]//2)]
    #sigma
    bs=np.zeros(k)
    for j in range(k):
        for i in range(int(v[j])):
            bs[j]+=abs(ss[j][i]-mus[j])
        bs[j] /= v[j]
        
    for j in range(k):
        sigmas[j] = math.sqrt(math.pi/2)*bs[j]
    
    """#SEM
    mus = np.zeros(k)
    for j in range(k):
        for i in range(n):
            if (ys[i,j] == 1):
                mus += data[i]
    mus /= v
    #sigma
    sigmas = np.zeros(k)
    for j in range(k):
        for i in range(n):
            if (ys[i,j] == 1):
                sigmas[j]+= (data[i]-mus[j])**2
    sigmas /= v
    sigmas = np.sqrt(sigmas)
    """
    print(pis)
    print(mus)
    print(sigmas)

EM iteration 0
[49.0000000000 113.0000000000 74.0000000000 110.0000000000 150.0000000000
 53.0000000000 45.0000000000 157.0000000000 154.0000000000 135.0000000000]
0
1
5
6
7
9
[0.0471153846 0.1086538462 0.0711538462 0.1057692308 0.1442307692
 0.0509615385 0.0432692308 0.1509615385 0.1480769231 0.1298076923]
[0.2800000000 -0.1200000000 0.1300000000 -0.0950000000 0.0200000000
 0.2700000000 0.5000000000 0.0400000000 0.0850000000 0.3800000000]
[1.3308149911 1.4463023319 1.8935205480 2.1652710787 1.2143778448
 1.6747114567 1.1683672902 0.5449122485 1.9544375979 2.3274507720]
EM iteration 1
[49.0000000000 139.0000000000 67.0000000000 112.0000000000 138.0000000000
 46.0000000000 41.0000000000 146.0000000000 165.0000000000 137.0000000000]
0
1
2
6
8
9
[0.0471153846 0.1336538462 0.0644230769 0.1076923077 0.1326923077
 0.0442307692 0.0394230769 0.1403846154 0.1586538462 0.1317307692]
[0.5100000000 -0.2300000000 0.2500000000 0.2900000000 0.1050000000
 0.3150000000 0.4900000000 0.0650000000 -0.4000

In [95]:
ss.sum(0)/n

array([0.0961538462, 0.1144230769, 0.1096153846, 0.0961538462,
       0.0817307692, 0.0894230769, 0.0865384615, 0.1201923077,
       0.1028846154, 0.1028846154])

In [96]:
n

1040

In [90]:
sigmas

array([1.0874789270, 1.6510121138, 1.6408800424, 1.2063566207,
       1.1770543303, 1.8068250238, 1.0018698205, 0.6163418853,
       2.0169134835, 4.0124983111])

In [64]:
a = ws[:,0]/sum(ws[:,0])

In [85]:
ws[0]

array([0.0000000005, 0.0000006516, 0.0000211332, 0.0000508978,
       0.0000000032, 0.0000016444, 0.0000003665, 0.0000000000,
       0.0001199896, 0.0002829505])

In [62]:
np.random.multinomial(1, a, size=1)

array([[0.0000000000, 0.0000000000, 0.0000000000, 0.0000000000,
        0.0000000000, 0.0000000000, 0.0000000000, 0.0000000000,
        0.0000000000, 1.0000000000]])